<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/VoiceChatGPT_API_CSROOM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT API + Whisper + gTTs

## ライブラリのインストール

In [ ]:
!pip install openai gTTs pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## ライブラリのインポート

In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from gtts import gTTS
import IPython.display
from pydub import AudioSegment
import openai
openai.api_key = ""#@param{type:"string"}

## 音声認識の準備

In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

In [ ]:
RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
  })
  var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
    recorder.stop()
  })
"""
def speech_to_text(model='whisper-1', language='ja', second=5):
  filename='tmp.wav'
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (second * 1000))
  b = b64decode(s.split(',')[1])

  with open(filename, 'wb+') as fw:
    fw.write(b)

  with open(filename, "rb") as fr:
    transcription = openai.Audio.transcribe(
        model=model,
        file=fr,
        language=language
    )
    return transcription['text']

## ChatGPT と会話する

### 人格、役割などの設定

In [ ]:
system_messages = [
    {'role': 'system', 'content': 'あなたは有能なアシスタントです。'},
    {'role': 'system', 'content': '質問にはなるべく短く返答してください。'}
]

### 質疑応答の例（いろいろ試してください）

In [ ]:

IPython.display.Audio("audio.wav", autoplay=True)

In [ ]:

sound = AudioSegment.from_file("audio.wav")
sound.export("audio.mp3", format="mp3")

<_io.BufferedRandom name='audio.mp3'>

In [ ]:
second = 5
print("質問して下さい（５秒）")
record(second, "audio.wav")

from pydub import AudioSegment
sound = AudioSegment.from_file("audio.wav")
sound.export("audio.mp3", format="mp3")

audio_file= open("audio.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
question = transcript["text"]
print("質問：", question, end="")
user_messages=[{
    "role": "user",
    "content": question
}]

print("\n回答：", end="")
messages = system_messages + user_messages
result = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
    stream = True,
)
response_text = ""
for chunk in result:
    content = chunk["choices"][0].get("delta", {}).get("content")
    if content is not None:
        print(content, end='', flush=True)
        response_text += content
print("")

message = response_text
tts = gTTS(message, lang='ja')
tts.save('gTTS_test.mp3')
IPython.display.display(IPython.display.Audio('gTTS_test.mp3', autoplay=True))

質問して下さい（５秒）


<IPython.core.display.Javascript object>

質問： こんにちは。自己紹介をしてください。
回答：こんにちは！私はAIのアシスタントです。日々学習しており、お手伝いができる範囲で質問に答えます。何かお力になれることがあればぜひお聞かせください。


## 繰り返し（作成中）

In [ ]:
messages = system_messages # 新しくリストを定義する
max_messages = 10 # 最大メッセージ数を定義する

while True:
    print("喋る準備ができたらエンターキーを押してください。")
    input()
    print("質問して下さい（５秒）")
    question = speech_to_text()
    print("質問：", question)
    messages.append({"role": "user", "content": question})

    if 'メッセージ' in question:
        print('messages:', messages)
        continue

    print("\n回答：", end="")
    result = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages,
        stream = True,
    )
    response_text = ""
    for chunk in result:
        content = chunk["choices"][0].get("delta", {}).get("content")
        if content is not None:
            print(content, end='', flush=True)
            response_text += content
    print("")
    messages.append({"role": "assistant", "content": response_text})

    tts = gTTS(response_text, lang='ja')
    tts.save('gTTS_test.mp3')
    IPython.display.display(IPython.display.Audio('gTTS_test.mp3', autoplay=True))

    if len(messages) > max_messages * 2 + 1:
        messages.pop(1)
        messages.pop(1)

    if '終了' in question:
        break